In [1]:
import tensorflow as tf
import numpy as np


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)


In [4]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
    # [batch_size, image_width, image_height, channels]
    # because the images are black, we only have one channel
    # -1 for batch_size means that the dimension should be calculated dynamically based on the number of input values in features["x"]
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
    # the activation map / tensor produced by this convolutional layer is [batch_size,28,28,32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer, # input tensor with the shape [batch_size, image_width, image_height, channels]
      filters=32, # number of filters to apply
      kernel_size=[5, 5], # dimensions of the filter
      padding="same", # keeps the activation map the same widht and height as the input, adding 0 values as padding, without padding it will be 24x24
      activation=tf.nn.relu)

  # Pooling Layer #1
    # the input will be conv1 with a shape of [batch_size,28,28,32] 
    # the pooling filter is 2x2 and the stride is 2 so that the regions do not overlap
    # the output of the pooling layer will be [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 
    # a second convolutional layer with double the nr of filters
    # the input has a shape of [batch_size, 14, 14, 32]
    # and the output has a shape of [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
    # the output of pool2 is [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer / fully connected
    # here we flatten our feature map to [batch_size, features] so that we have only 2 dimentsions
    # the output tensor will have a shape of [batch_size, 3136] . 3136 from  7 * 7 * 64 
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64]) 
    
    # the input specifies our input tensor 
    # the units are the number of our neurons the dense layer
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    # dropout regularization -> good to prevent overfitting 
    # 0.4 rate means that 40% of the elements will be randomly dropped out during training
    # the network will become smaller and it will be easier to train
    # the output here will have a shape of [batch_size, 1024]
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
    # it outputs the values that are going to be the input of the softmax function
    # this is the final layer that will return the raw values for our predictions
    # it has 10 neurons, one for each target class
    # it uses linear activation which is the default
    # the output has a shape [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  # we put our predictions in a dictionary
  predictions = {
      # Generate predictions (for PREDICT and EVAL (evaluation) mode)
      # for example, is it a 1, a 3 etc. from 0 to 9
      # we use argmax to extract the highest raw value from the logits tensor
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      # the probabilities that it is a 1,3, 5 etc.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  
  # and return an EstimatorSpec object from our dict
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL mode
  # see how closely the model's predictions match the target classes
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  # optimize the loss during training with stochastic descent
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode), like accuracy
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [9]:
# train and test data
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator, which is the main thing that trains and evaluates
    # we chose the model cnn_model_fn that we created above
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=600,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)



In [10]:
unused_argv = {} # no arguments for now
main(unused_argv)

Extracting MNIST-data/train-images-idx3-ubyte.gz


Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': './logs/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x110b5d7b8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Starting evaluation at 2018-08-15-14:27:56


INFO:tensorflow:Restoring parameters from ./logs/model/model.ckpt-600


INFO:tensorflow:Finished evaluation at 2018-08-15-14:28:01


INFO:tensorflow:Saving dict for global step 600: accuracy = 0.4353, global_step = 600, loss = 2.22162


{'accuracy': 0.43529999, 'loss': 2.2216158, 'global_step': 600}
